In [36]:
MAIN_DIRECTORY = "./assets/ETL9G"
OUTPUT_DIRECTORY = "./assets/ETL9G_OUTPUT"

A single CSV file contains multiple images and multiple txt files. For that reason, use csv file names as a standard

In [9]:
import os
csv_directory = os.listdir(MAIN_DIRECTORY + "/csv")
ETL9G_FILES = [file_name.split(".")[0] for file_name in csv_directory]
ETL9G_FILES.sort()

In [13]:
def retrieve_images(file_number):
  img_directory = os.listdir(MAIN_DIRECTORY + "/png")
  img_directory = [file_name for file_name in img_directory if file_name.startswith(file_number)]
  return img_directory

In [ ]:
def retrieve_text(file_number):
  txt_directory = os.listdir(MAIN_DIRECTORY + "/txt")
  txt_directory = [file_name for file_name in txt_directory if file_name.startswith(file_number)]
  return txt_directory

In [74]:
from PIL import Image
def split_images(image_file_name):
  file_name = image_file_name.split(".")[0]
  with open(MAIN_DIRECTORY + "/txt/" + file_name + ".txt", "r") as file:
    lines = file.readlines()
  lines = [line.split("\n")[0] for line in lines]
  height = len(lines)
  width = len(lines[0])
  image = Image.open(MAIN_DIRECTORY + "/png/" +  image_file_name)
  image_width, image_height = image.size
  tile_width = image_width // width
  tile_height = image_height // height
  
  tiles = []
  
  for row in range(height):
    tile_row = []
    for col in range(width):
      left = col * tile_width
      upper = row * tile_height
      right = left + tile_width
      lower = upper + tile_height
      
      cropped = image.crop((left, upper, right, lower))
      tile_row.append(cropped)
    tiles.append(tile_row)
  lines = [list(line) for line in lines]
  return (tiles, lines)
  

In [60]:
import csv
class Kanji:
    def __init__(self, serial_sheet, jis_code, jis_reading, serial_data, evaluation_image, 
                 evaluation_group, male_female, age_writer, industry_classification, 
                 gathering_date, scanning_date, pos_x, pos_y):
      self.serial_sheet = serial_sheet
      self.jis_code = jis_code
      self.jis_reading = jis_reading
      self.serial_data = serial_data
      self.evaluation_image = evaluation_image
      self.evaluation_group = evaluation_group
      self.male_female = male_female
      self.age_writer = age_writer
      self.industry_classification = industry_classification
      self.gathering_date = gathering_date
      self.scanning_date = scanning_date
      self.pos_x = pos_x
      self.pos_y = pos_y
    
def read_csv(csv_file_name):
  Kanjis = []
  with open(MAIN_DIRECTORY + "/csv/" + csv_file_name) as file:
    reader  = csv.DictReader(file)
    for row in reader:
      kanji = Kanji(
          serial_sheet=int(row['Serial Sheet Number']),
          jis_code=row['JIS Kanji Code'],
          jis_reading=row['JIS Typical Reading'],
          serial_data=row['Serial Data Number'],
          evaluation_image=row['Evaluation of Individual Character Image'],
          evaluation_group=row['Evaluation of Character Group'],
          male_female=row['Male-Female Code'],
          age_writer=int(row['Age of Writer']),  # Assuming 'age_writer' is numeric
          industry_classification=row['Industry Classification Code'],
          gathering_date=row['Sheet Gatherring Date'],  # Ensure it's in the correct format (date parsing can be done if needed)
          scanning_date=row['Scanning Date'],  # Ensure it's in the correct format (date parsing can be done if needed)
          pos_x=int(row['Sample Position X on Sheet']),  # Assuming x and y are integers
          pos_y=int(row['Sample Position Y on Sheet'])   # Same for y position
          )
      Kanjis.append(kanji)
  return Kanjis

In [ ]:
images = retrieve_images(ETL9G_FILES[0])
for image in images:
  tiles, lines = split_images(image)
  for i in range(len(lines)):
    for j in range(len(lines[i])):
      tiles[i][j].save(OUTPUT_DIRECTORY + "/" + lines[i][j] + ".png")